In [123]:
import os
import numpy as np
import random
from PIL import Image
import matplotlib.pyplot as plt
from math import floor
import pandas as pd 
import cv2
import matplotlib.gridspec as gridspec
import tensorflow as tf
import tensorflow.keras as keras
import tensorflow.keras.layers as layers
import random
from keras.initializers import RandomNormal
from keras.optimizers import Adam
from numpy.random import randn

In [124]:
original = []

for filename in os.listdir('/kaggle/input/gan-getting-started/monet_jpg'):
    original.append(np.array(Image.open('/kaggle/input/gan-getting-started/monet_jpg/'+str(filename)).resize((128,128))))
    #original.append(np.array(Image.open('/kaggle/input/gan-getting-started/monet_jpg/'+str(filename))))
    
print(np.array(original).shape)
original = (np.array(original)- 127.5) / 127.5  #normalize the input images to speed up the training




In [125]:
def Plot(arr):
    arr = (arr + 1) / 2.0
    fig, axes = plt.subplots(2, 2)
    fig.set_size_inches(16,8)
    count = 0
    for i in range(2):
        for j in range(2):
            c = arr[count].copy()
            c.resize((256,256))
            axes[i, j].imshow(arr[count])
            axes[i, j].axis('off')
            count += 1
    plt.tight_layout() 
    plt.show()
    

In [126]:
Plot(original)

In [127]:
latent_dim =100
height = 128
width = 128

channels = 3

In [128]:
def Generator():
  
    generator_input = keras.Input(shape = (latent_dim,))
    
    init= RandomNormal(mean=0.0, stddev=0.02)
    gamma_init =RandomNormal(mean=1, stddev=0.02)
    
    x = layers.Dense(256 * 1 * 1,kernel_initializer=init)(generator_input)
    x = layers.Reshape((1,1,256))(x)
    x = layers.LeakyReLU()(x)
 
 
    #x = layers.Conv2D(32, 3,strides = 2, padding='same')(x) #stride b 2 to downsample instead of max pooling.
    #x = layers.LeakyReLU()(x)

    #x = layers.Conv2D(64, 3,strides = 2, padding='same')(x)
    #x = layers.LeakyReLU()(x)
  
  
    x = layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x) #stride 2 with covTranspose to upsample.
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)   
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)

    x = layers.Conv2DTranspose(256, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2DTranspose(128, (4,4), strides=(2,2), padding='same',kernel_initializer=init)(x)
    x = layers.BatchNormalization(gamma_initializer=gamma_init)(x)
    x = layers.LeakyReLU()(x)
    
  
  
 
    x = layers.Conv2DTranspose(channels, 3, activation = 'tanh', padding = 'same')(x)  # tanh to get values between 1 and -1 same as monet images

    generator = keras.models.Model(generator_input, x)

    generator.summary()

    return generator

In [129]:
def Discrimnator():
    discriminator_input = layers.Input(shape = (height, width, channels))
    init= RandomNormal(mean=0.0, stddev=0.02)
    
    x = layers.Conv2D(64, 4,kernel_initializer=init)(discriminator_input)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
    
    x = layers.Conv2D(128, 4, strides = 2,kernel_initializer=init)(x)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU(0.2)(x)
  
    x = layers.Conv2D(128, 4, strides = 2,kernel_initializer=init)(x)
    #x = layers.BatchNormalization()(x)
    x = layers.LeakyReLU()(x)
  
    x = layers.Flatten()(x)
  
    x = layers.Dense(1, activation = 'sigmoid')(x)
  
    discriminator = keras.models.Model(discriminator_input, x)
    discriminator.summary()

    return discriminator


In [130]:
generator=Generator()
discriminator=Discrimnator()

In [131]:
generator_optimizer = Adam(lr=0.0002, beta_1=0.5)
#generator.compile(optimizer = generator_optimizer, loss = 'categorical_crossentropy')

discriminator_optimizer = Adam(lr=0.0002, beta_1=0.5)
#discriminator.compile(optimizer = discriminator_optimizer,loss='binary_crossentropy')

cross_entropy = tf.keras.losses.BinaryCrossentropy(from_logits=False)

In [132]:
def Get_Noise_Batch():
    random_latent_vectors = randn(latent_dim * batch_size)
    # update to have the range [-1, 1]
    random_latent_vectors = -1 + random_latent_vectors * 2
    random_latent_vectors = random_latent_vectors.reshape((batch_size, latent_dim))
    
    return random_latent_vectors

In [133]:
def Get_True_Batch(idx):
    real_images=original[idx:idx+batch_size]
    return real_images

In [134]:
def G_loss(fake_output):
     return cross_entropy(tf.ones_like(fake_output), fake_output)

In [135]:
def D_loss(real_output,fake_output):
    real_loss = cross_entropy(tf.ones_like(real_output), real_output)
    fake_loss = cross_entropy(tf.zeros_like(fake_output), fake_output)
    return real_loss+fake_loss

In [136]:
gen_loss_total=[]
disc_loss_total=[]

In [140]:
def step(i,idx,batch_size=30):
    

    noise=Get_Noise_Batch()
    real=Get_True_Batch(idx)

    with tf.GradientTape() as gen_tape, tf.GradientTape() as disc_tape:
        generated_images = generator(noise, training=True)
        
        real_output = discriminator(real, training=True)
        fake_output = discriminator(generated_images, training=True)
        
        gen_loss = G_loss(fake_output)
        disc_loss =D_loss(real_output,fake_output)

        
 
    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    gradients_of_discriminator = disc_tape.gradient(disc_loss,discriminator.trainable_variables)

    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    discriminator_optimizer.apply_gradients(zip(gradients_of_discriminator,discriminator.trainable_variables))
    
    
    if i%200==0 or i==29999 :
        print("Iteration= ",i," results:")   
        print('discriminator loss:', disc_loss)
        print('generator loss:', gen_loss)
        gen_loss_total.append(gen_loss)
        disc_loss_total.append(disc_loss)
      
        Plot(generated_images.numpy())
    
   

In [ ]:
iterations = 30000
batch_size = 60
idx = 0

for i in range(iterations):
    
    if idx<300:
        step(i,idx,batch_size=30)
    else:
        idx=0
        step(i,idx,batch_size=30)
        
    idx=idx+batch_size



In [ ]:
plt.plot(list(range(0, iterations+1,200)),gen_loss_total, list(range(0, iterations+1,200)),disc_loss_total)
plt.legend(['gen loss','disc loss'])
plt.xlabel("iterations")
plt.ylabel("loss values")
plt.show()